# Lemke Howson Algorithm Final Project
EN.605.726/625.741 Game Theory

Tom O'Brien and Joe Dimino

The focus of this project is a Python 2 implementation of the Lemke Howson Algorithm using the tableaux method.  This project additionally has implemented a preprocessing function using Iterative Elimination of Strictly Dominated Pure Strategies and a function for verification of output Nash Equilibrium strategies through the Characterization of Mixed Strategy Nash Equilibrium Theorem.

The project is divided in to five sections below:
1. Make a Game
2. Preprocessing
3. Lemke Howson Algorithm
4. Verification of Nash Equilibrium
5. Testing

## 1. Make a Game:

In [1]:
import numpy as np
def read_game(filename):
    A = []
    B = []
    with open(filename) as textFile:
        strategies = [line.split() for line in textFile]
        for i in range(len(strategies)):
            A.append([])
            B.append([])
            for j in range(len(strategies[i])):
                p= strategies[i][j].strip("()").split(",")
                x = float(p[0])
                y = float(p[1])
                A[i].append(x)
                B[i].append(y)
    return (np.matrix(np.array(A)), np.matrix(np.array(B)))      

            
    

In [2]:
def random_normalized_vector(row, column):
    v = np.random.rand(row, column)
    return (v / sum(v))

In [3]:
def expected_payoff(player, x, y, A, B):
    x = np.matrix(x)
    y = np.matrix(y)
    if player == 1:
        p = A
    elif player == 2:
        p = B
        
    return np.array(x*p*y.transpose())[0][0]
    
    
    

## 2. Preprocessing

### Iterative Elimination of Strictly Dominated Pure Strategies

In [5]:
def strictly_dominates(player, strat1, strat2, A, B):
    A = np.array(A)
    B = np.array(B)
    dominates = True
    (m,n) = np.shape(A)
    if player == 1:
        for j in range(n):
            if A[strat1][j] <= A[strat2][j]:
                dominates = False
    if player == 2:
        for i in range(m):
            if B[i][strat1] <= B[i][strat2]:
                dominates = False
    return dominates
            

In [6]:
def remove_dominated_strategies(player, strategies, A, B):
    return (np.delete(A, strategies, axis=((player+1) % 2)), np.delete(B, strategies, axis=((player+1) % 2)))           
          

In [4]:
def eliminate_strictly_dominated_pure_strategies(A,B):
    (m0, n0) = np.shape(A)
    remaining_p1_strategies = range(m0)
    remaining_p2_strategies = range(n0)
    cycles_no_strictly_dominated = 0
    player = 1
    while(cycles_no_strictly_dominated < 2):
        #print "Looking for player " + str(player) + " dominated pure strategies"
        (m,n) = np.shape(A) # will change based on removed strategies
        strictly_dominated_exist = False # will be changed to true if one strictly dominated strategy is found in nested loop
        dominated_strategies = []
        for i in range(m):
            for k in range(m):
                if i != k and strictly_dominates(player, i, k, A, B):
                    #print str(i) + " dominates " + str(k) + " for player " + str(player)
                    strictly_dominated_exist = (True)
                    cycles_no_strictly_dominated = 0
                    if k not in dominated_strategies:
                        dominated_strategies.append(k)
        dominated_strategies = sorted(dominated_strategies, reverse = True)
        #print "Dominated strategies found: " + str(dominated_strategies)
        for d in dominated_strategies:
            if player == 1:
                #print "Removing " + str(remaining_p1_strategies[d]) + " for Player 1"
                del remaining_p1_strategies[d]
            if player == 2:
                #print "Removing " + str(remaining_p2_strategies[d]) + " for Player 2"
                del remaining_p2_strategies[d]
        if strictly_dominated_exist == False:
            cycles_no_strictly_dominated = cycles_no_strictly_dominated + 1
            #print "No dominated strategies found for " + str(cycles_no_strictly_dominated) + " cycles"
        (A,B) = remove_dominated_strategies(player,dominated_strategies, A, B)
        player = (player % 2) + 1
        #print "Player changed to " + str(player)
        #print A
        #print B
        #print remaining_p1_strategies
        #print remaining_p2_strategies
        #print m0
        #print n0
    return [(A,B), remaining_p1_strategies, remaining_p2_strategies, m0, n0]
                    
        
    

# 3. Lemke Howson Algorithm

In [31]:
def min_ratio(tableaux, column):
    (rows,cols) = np.shape(tableaux)
    tableaux = np.array(tableaux)
    ratios = []
    for i in range(rows):
        ratios.append(tableaux[i][column])
    val, idx = min((val, idx) for (idx, val) in enumerate(ratios))
    return idx

    

In [32]:
def find_next_label(k, r, s):
    (m,r_columns) = np.shape(r)
    (n,s_columns) = np.shape(s)
    tableaux = None
    if k < m:
        tableaux = s
        column = k+1
        return min_ratio(tableaux, column) + m
    if k >= m:
        tableaux = r
        column = k-m+1
        return min_ratio(tableaux, column)

In [33]:
def perform_pivot_math(tableaux, row, column):
    (rows, cols) = np.shape(tableaux)
    tableaux = np.array(tableaux)
    tableaux[row][row+rows+1] = tableaux[row][row+rows+1] - 1 #subtract 1 to move coeffient of the s variable to right side
    coefficient = tableaux[row][column]*-1
    for c in range(cols):
        tableaux[row][c] = tableaux[row][c]/coefficient
    tableaux[row][column] = 0
    sub = tableaux[row]
    for my_row in range(rows):
        temp = tableaux[my_row]
        tableaux[my_row]= temp+sub*tableaux[my_row][column]
        tableaux[my_row][column] = 0
    return np.matrix(tableaux)

In [34]:
def pivot(next_label, label, r, s):
    (m,r_cols) = np.shape(r)
    (n,s_cols) = np.shape(s)
    tableaux = None
    row = None
    column = None
    if label >= m:
        row = next_label
        column = label-m+1
        r = perform_pivot_math(r, row, column)
    else:
        row = next_label -m
        column = label + 1
        s = perform_pivot_math(s, row, column)
    return (r,s)        

    

In [35]:
def extract_nash(label_order, r, s):
    (m, r_cols) = np.shape(r)
    (n, s_cols) = np.shape(s)
    r = np.array(r)
    s = np.array(s)
    variable = {}
    for i in range(len(label_order)):
        if i != len(label_order)-1:
            variable[label_order[i]] = label_order[i+1]
        if i == len(label_order)-1:
            variable[label_order[i]] = label_order[0]
        
    x = np.zeros(m)
    y = np.zeros(n)
    for i in variable:
        if i < m:
            xi = s[variable[i]-m][0]
            x[i] = xi
        else:
            yi = r[variable[i]][0]
            y[i-m] = yi
            
    x = x/np.sum(x)
    y = y/np.sum(y)
    return (x,y)


In [36]:
def lemke_howson(A,B):
    #new_game = eliminate_strictly_dominated_pure_strategies(A,B)
    #(A,B) = new_game[0]
    #remaining_p1_strategies = new_game[1]
    #remaining_p2_strategies = new_game[2]
    #m0 = new_game[3]
    #n0 = new_game[4]
    (m,n) = np.shape(A)
    #Initialize tableaux r [column of 1's for real number, -Ay, mxm matrix of zeros for coefficients of r1, r2, ... rm]
    r = np.hstack([np.matrix(np.ones((m,1),dtype=float)),-A,np.matrix(np.zeros((m,m),dtype=float))])
    #Initialize tableaux s [column of 1's for real number, -B'x, nxn matrix of zeros for coefficients of s1, s2, ...sn]
    s = np.hstack([np.matrix(np.ones((n,1),dtype=float)),-B.transpose(), np.matrix(np.zeros((n,n),dtype=float))])
    labels = []
    k0 = np.random.randint(0,m)
    print "\nK0 = " + str(k0)
    k=k0
    iterations = 0
    
    print "Initial R tableaux:"
    print r
    print "\nInitial S tableaux:"
    print s
    print "Entering pivot loop..."
    print "\n\n"
        
    while(iterations < 1 or k != k0): #iterations added to allow it to go through initial iteration  
        labels.append(k)
        iterations = iterations+1
        new_k = find_next_label(k, r, s)
        (r,s) = pivot(new_k, k, r, s)
        k = new_k
       
        print "R tableaux:"
        print r
        print "\nS tableaux:"
        print s
        print "Next label is " + str(k)
        print "\n\n"
    
    print labels
    print "Pivot loop exited. Extracting Nash Equilibrium..."
    
    
    return extract_nash(labels, r, s)
    
    
    
    
    
        
    

## 4. Verification of Nash Equilibrium

In [42]:
def find_supports(vector, epsilon):
    supports = []
    nonsupports = []
    vector = list(vector)
    for i in range(len(vector)):
        if vector[i] > epsilon:
            supports.append(i)
        else:
            nonsupports.append(i)
    return (supports, nonsupports)   

In [39]:
def verify_nash(x,y,A,B,epsilon):
    (m,n) = np.shape(A)
    (x_supports, x_nonsupports) = find_supports(x,epsilon)
    (y_supports, y_nonsupports) = find_supports(y,epsilon)
    
    E_x = [] #expected payoffs for x supports
    E_y = [] #expected payoffs for y supports
    E_nx = [] #expected payoffs for x nonsupports
    E_ny = [] #expected payoffs for y nonsupports
    
    for s in x_supports:
        xi = np.zeros(m)
        xi[s] = 1
        E_x.append(expected_payoff(1,xi,y,A,B))
    print "E_x"
    print E_x
    
    for s in y_supports:
        yi = np.zeros(n)
        yi[s] = 1
        E_y.append(expected_payoff(2,x,yi,A,B))
    print "\nE_y"
    print E_y
    
    for non in x_nonsupports:
        xi = np.zeros(m)
        xi[non] = 1
        E_nx.append(expected_payoff(1,xi,y,A,B))
    print "\nE_nx"
    print E_nx
    
    for non in y_nonsupports:
        yi = np.zeros(n)
        yi[non] = 1
        E_ny.append(expected_payoff(2,x,yi,A,B))
        
    print "\nE_ny"
    print E_ny            
    
    if not all(abs(t-E_x[0]) <= epsilon for t in E_x): return False
    if not all(abs(t-E_y[0]) <= epsilon for t in E_y): return False
    if not all(t < E_x[0] for t in E_nx): return False
    if not all(t < E_y[0] for t in E_ny): return False
    
    return True
    


## 5. Testing

### Game Definitions

In [7]:
Battle_of_the_Sexes = '/home/tom/Desktop/Game Theory Final Project/battle_of_sexes.txt'
Tableaux = '/home/tom/Desktop/Game Theory Final Project/Tableaux Example'
Dominance_Solvable_Example = '/home/tom/Desktop/Game Theory Final Project/Dominance Solvable'

In [30]:
BoS = read_game(Battle_of_the_Sexes)
Tab = read_game(Tableaux)
Dom = read_game(Dominance_Solvable_Example)


In [37]:
(A,B) = Tab
(x,y) = lemke_howson(A,B)
print (x,y)


K0 = 2
Initial R tableaux:
[[ 1. -1. -3. -0.  0.  0.  0.]
 [ 1. -0. -0. -2.  0.  0.  0.]
 [ 1. -2. -1. -1.  0.  0.  0.]]

Initial S tableaux:
[[ 1. -2. -1. -0.  0.  0.  0.]
 [ 1. -1. -3. -0.  0.  0.  0.]
 [ 1. -0. -1. -3.  0.  0.  0.]]
Entering pivot loop...



R tableaux:
[[ 1. -1. -3. -0.  0.  0.  0.]
 [ 1. -0. -0. -2.  0.  0.  0.]
 [ 1. -2. -1. -1.  0.  0.  0.]]

S tableaux:
[[ 1.         -2.         -1.          0.          0.          0.          0.        ]
 [ 1.         -1.         -3.          0.          0.          0.          0.        ]
 [ 0.33333333 -0.         -0.33333333  0.          0.          0.
  -0.33333333]]
Next label is 5



R tableaux:
[[ 1.  -1.  -3.   0.   0.   0.   0. ]
 [ 0.5 -0.  -0.   0.   0.  -0.5  0. ]
 [ 0.5 -2.  -1.   0.   0.   0.5  0. ]]

S tableaux:
[[ 1.         -2.         -1.          0.          0.          0.          0.        ]
 [ 1.         -1.         -3.          0.          0.          0.          0.        ]
 [ 0.33333333 -0.         -0.

In [40]:
verify_nash(x,y,A,B,.00001)

E_x
[1.1111111111111112, 1.1111111111111112, 1.1111111111111112]

E_y
[1.153846153846154, 1.1538461538461537, 1.1538461538461537]

E_nx
[]

E_ny
[]


True